In [ ]:
# Copyright Chenyao2333

In [3]:
def to_two_complement(n):
    if (n < 0):
        n = (-n) ^ 0xFF
        n += 1
    
    return n

def to_hex(n):
    return hex(n)[2:].upper()

def to_dec(n):
    return int(n, 16)

S = 34*256 + 255
print(to_hex(S))
print(to_dec("8"))

print(to_hex(((1) ^ 0xFF) + 1))

22FF
8
FF


In [4]:
print(int("0x10"))

ValueError: invalid literal for int() with base 10: '0x10'

In [5]:
TEST_VAR_TABLE = {
    "i": 1,
    "start": 10
}

def is_dec(s):
    for i in s:
        if not i.isdigit():
            return False
    return True

def is_hex(s):
    if not s.startswith("0x"):
        return False
    try:
        int(s, 16)
        return True
    except:
        return False

def parse_var(v, var_table):
    if isinstance(v, int):
        return v
    elif v == "":
        return 0
    elif v in var_table:
        return var_table[v]
    elif is_hex(v):
        return int(v, 16)
    elif is_dec(v):
        return int(v)
    elif "+" in v:
        return sum(list(map(lambda x: parse_var(x, var_table), v.split("+"))))
    elif "-" in v:
        p = v.rfind("-")
        return parse_var(v[0:p], var_table) - parse_var(v[p+1:], var_table)
    else:
        raise Exception()

In [6]:
var = ["11-i", "0x46", "0xA+i-start-1", "start+1-0x1+1"]
for i in var:
    print(parse_var(i, TEST_VAR_TABLE))

10
70
0
11


In [13]:
class Instruction(object):
    OP2CODE = {
        "load-memory": 1,
        "load-pattern": 2,
        "store": 3,
        "add": 5,
        "and": 8,
        "jump": 11,
    }
    OP_OPERAND = {
        "load-memory": "48",
        "load-pattern": "48",
        "store": "48",
        "add": "444",
        "and": "444",
        "jump": "48",
    }
    def __init__(self, operation, operand = []):
        print ("op = %s operand = %s" % (operation, operand))
        if (len(operand) != len(self.OP_OPERAND[operation])):
            raise Exception("Incorrect operation: %s %s" % (operation, operand))

        self.operation = operation
        self.operand = list(operand)

    def process_variable(self, var_table):
        for i in range(0, len(self.operand)):
            try:
                self.operand[i] = parse_var(self.operand[i], var_table)
            except:
                print("Failing to parse %s" % self.operand[i])
                pass
            
    def to_machine_code(self, var_table):
        n = self.OP2CODE[self.operation]
        pat = self.OP_OPERAND[self.operation]

        for i in range(0, len(self.operand)):
            # print(self.operand[i])
            n *= 2**to_dec(pat[i])
            n += to_two_complement(parse_var(self.operand[i], var_table))
        
        #print(n)
        return to_hex(n)

In [14]:
ins = Instruction("jump", ["i", "start"])

op = jump operand = ['i', 'start']


In [15]:
ins.to_machine_code(TEST_VAR_TABLE)

'B10A'

In [16]:
class MachineCode(object):
    def __init__(self, src):
        self.instructions = []
        self.var_table = {}
        
        for line in src.splitlines():
            line = line.strip()
            if line.startswith("#"):
                continue
            ops = line.split()
            if len(ops) == 0:
                continue
            
            if ops[0] == "using":
                self.var_table[ops[1]] = int(ops[-1])
            elif ops[0] == "tag":
                self.var_table[ops[1]] = len(self.instructions) * 2
            else:
                ins = Instruction(ops[0], ops[1:])
                ins.process_variable(self.var_table)
                self.instructions.append(ins)
            
        self.var_table["__code_length"] = len(self.instructions) * 2
            
    def __repr__(self):
        s = ""
        for i in self.instructions:
            s += i.to_machine_code(self.var_table) + "\n"
        return s

In [17]:
code = """
using i           register 1
using minus1      register 2
using tmp         register 3
using zero        register 0
using to          register 5

load-pattern minus1 -1
load-pattern i __code_length
tag start
    add i i minus1
    
    # read from memory into tmp
    store i load_pos+1
    tag load_pos
    load-memory tmp 0xAA
    
    # write back to memory with shift 0x70
    load-pattern to 0x70
    add to to i
    store to store_pos+1
    tag store_pos
    store tmp 0xAA
    
    jump i 0x70
jump zero start
"""

In [18]:
mc = MachineCode(code)
print(mc.var_table)
print(mc)

op = load-pattern operand = ['minus1', '-1']
op = load-pattern operand = ['i', '__code_length']
Failing to parse __code_length
op = add operand = ['i', 'i', 'minus1']
op = store operand = ['i', 'load_pos+1']
Failing to parse load_pos+1
op = load-memory operand = ['tmp', '0xAA']
op = load-pattern operand = ['to', '0x70']
op = add operand = ['to', 'to', 'i']
op = store operand = ['to', 'store_pos+1']
Failing to parse store_pos+1
op = store operand = ['tmp', '0xAA']
op = jump operand = ['i', '0x70']
op = jump operand = ['zero', 'start']
{'store_pos': 16, 'i': 1, 'minus1': 2, 'zero': 0, '__code_length': 22, 'tmp': 3, 'start': 4, 'load_pos': 8, 'to': 5}
22FF
2116
5112
3109
13AA
2570
5551
3511
33AA
B170
B004



In [21]:
code_c = """
using condition     register 0
using problem_start register 11
using tmp0          register 5
using tmp1          register 6
using load_mem_pos  register 7
using store_mem_pos register 8
using ins           register 9
using i             register 1
using one           register 4
using two           register 13
using n70           register 2
using zero          register 12

load-pattern one 1
load-pattern two 2
load-pattern i -2
load-pattern n70 0x70

# load-pattern problem_start 0

tag while
    add i i two
    
    # load instruction from memory to tmp0 and tmp1
    add load_mem_pos problem_start i
    store load_mem_pos load_tag_0+1
    tag load_tag_0
    load-memory tmp0 0xAA
    
    add load_mem_pos load_mem_pos one
    store load_mem_pos load_tag_1+1 
    tag load_tag_1
    load-memory tmp1 0xAA
    
    
    
    # check if the tmp1(address) needs to add 70
    
    # check if the ins is load-pattern problem_start
    # the problem_start is register 11
    # load-pattern condition 0x2B
    # jump ins plus_70
    
    # extract instruction
    load-pattern ins 0xF0
    and ins ins tmp0
    
    # check if the ins is jump
    load-pattern condition 0xB0
    jump ins plus_70
    
    # check if the ins is store
    load-pattern condition 0x30
    jump ins plus_70
    
    
    tag write_back
    # store the instructions into the destination
    add store_mem_pos problem_start i
    add store_mem_pos store_mem_pos n70
    store store_mem_pos store_tag_0+1
    tag store_tag_0
    store tmp0 0xAA
    
    add store_mem_pos store_mem_pos one
    store store_mem_pos store_tag_1+1
    tag store_tag_1
    store tmp1 0xAA

    load-pattern condition __code_length-2
    jump i break
jump condition while

tag break
add problem_start problem_start n70
jump condition 0x70

# function: add tmp1 with 0x70
tag plus_70
    add tmp1 tmp1 n70
jump condition write_back
"""

In [22]:
mc_c = MachineCode(code_c)
print(mc_c.var_table)
print(mc_c)

op = load-pattern operand = ['one', '1']
op = load-pattern operand = ['two', '2']
op = load-pattern operand = ['i', '-2']
op = load-pattern operand = ['n70', '0x70']
op = add operand = ['i', 'i', 'two']
op = add operand = ['load_mem_pos', 'problem_start', 'i']
op = store operand = ['load_mem_pos', 'load_tag_0+1']
Failing to parse load_tag_0+1
op = load-memory operand = ['tmp0', '0xAA']
op = add operand = ['load_mem_pos', 'load_mem_pos', 'one']
op = store operand = ['load_mem_pos', 'load_tag_1+1']
Failing to parse load_tag_1+1
op = load-memory operand = ['tmp1', '0xAA']
op = load-pattern operand = ['ins', '0xF0']
op = and operand = ['ins', 'ins', 'tmp0']
op = load-pattern operand = ['condition', '0xB0']
op = jump operand = ['ins', 'plus_70']
Failing to parse plus_70
op = load-pattern operand = ['condition', '0x30']
op = jump operand = ['ins', 'plus_70']
Failing to parse plus_70
op = add operand = ['store_mem_pos', 'problem_start', 'i']
op = add operand = ['store_mem_pos', 'store_mem_pos

In [130]:
a = 0X100

In [131]:
print(a)

256
